In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import math as math
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from keras.regularizers import l2
from PIL import Image

In [ ]:
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Dataset folders 
dataset_dir = '../input/flippeddataset2/training_data_final_divided_and_flipped2'
training_dir  = os.path.join(dataset_dir, 'Training')
validation_dir = os.path.join(dataset_dir, 'Validation')
#test_dir = os.path.join(dataset_dir, 'test')

In [ ]:
# Plot example images from dataset
labels = ['Species1',       # 0
          'Species2',       # 1
          'Species3',       # 2
          'Species4',       # 3
          'Species5',       # 4
          'Species6',       # 5
          'Species7',       # 6
          'Species8',       # 7
          ]  

In [ ]:
input_shape = (96, 96, 3)

In [ ]:
from keras.applications.efficientnet import preprocess_input, EfficientNetB4
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rotation_range=45,
                                        height_shift_range=0.3,
                                        width_shift_range=0.3,
                                        zoom_range=0.4,
                                        vertical_flip=True, 
                                        brightness_range=[0.2,1.2],
                                        fill_mode='reflect',
                                        preprocessing_function=preprocess_input)


train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=labels,
                                               class_mode='categorical',
                                               batch_size=16,
                                               shuffle=True,
                                               seed=seed)
valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)#, rescale=1/255.)

valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=labels,
                                               class_mode='categorical',
                                               batch_size=16,
                                               shuffle=False,
                                               seed=seed)

In [ ]:
from collections import Counter

counter = Counter(train_gen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}          

class_weights

In [ ]:
supernet = EfficientNetB4(include_top=False,
                        weights='imagenet',
                        input_shape=(380,380,3))

supernet.trainable = False

last_nonTrainable_layer = 10
for i, layer in enumerate(supernet.layers[:last_nonTrainable_layer]):
  layer.trainable=False

for i, layer in enumerate(supernet.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []
  """
  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)
  """
  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
               
  callbacks.append(es_callback)

  # reduce learning rate on plateau

  plat_callbacks=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-5)
  
  callbacks.append(plat_callbacks)           

  return callbacks

In [ ]:
# Use the supernet as feature extractor

inputs = tfk.Input(shape=input_shape)

x = tfk.layers.Resizing(380,380, interpolation='bilinear', crop_to_aspect_ratio=True)(inputs)

x = supernet(x)

glob_pooling = tfkl.GlobalAveragePooling2D(name='GlobalPooling')(x)

x = tfkl.Dense(
    512,
    kernel_initializer = tfk.initializers.GlorotUniform(seed),
)(glob_pooling)

leaky_relu_layer = tfkl.LeakyReLU()(x)

leaky_relu_layer = tfkl.BatchNormalization()(leaky_relu_layer)

x = tfkl.Dropout(0.2, seed=seed)(leaky_relu_layer)

y = tfkl.Dense(
    256,
    kernel_initializer = tfk.initializers.GlorotUniform(seed),
)(x)

leaky_relu_layer2 = tfkl.LeakyReLU()(y)

leaky_relu_layer2 = tfkl.BatchNormalization()(leaky_relu_layer2)

leaky_relu_layer2 = tfkl.Dropout(0.3, seed=seed)(leaky_relu_layer2)

outputs = tfkl.Dense(
    8, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed),
)(x)


# Connect input and output through the Model class
ft_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(),  
                 optimizer=tfk.optimizers.Adam(learning_rate=1e-2), 
                 metrics=['accuracy'])
ft_model.summary()

In [ ]:
callbacks = create_folders_and_callbacks(model_name='transferLearningModel')

tl_history = ft_model.fit(
    x = train_gen,
    batch_size = 16,
    epochs = 200,
    validation_data = valid_gen,
    class_weight = class_weights,
    callbacks = callbacks
).history



In [ ]:
ft_model.save("efficientNetB4_tl")

In [ ]:
supernet.trainable = True

In [ ]:
last_nonTrainable_layer = 185
for i, layer in enumerate(supernet.layers[:last_nonTrainable_layer]):
  layer.trainable=False

for i, layer in enumerate(supernet.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(),  
                 optimizer=tfk.optimizers.Adam(learning_rate=1e-4), 
                 metrics=['accuracy'])

In [ ]:
callbacks = create_folders_and_callbacks(model_name='fineTunedModel')

ft_history = ft_model.fit(
    x = train_gen,
    batch_size = 16,
    epochs = 200,
    validation_data = valid_gen,
    class_weight = class_weights,
    callbacks = callbacks
).history


In [ ]:
ft_model.save("efficientNetB4_ft")

In [ ]:
def get_next_batch(generator):
    return next(generator)

In [ ]:
for i in range(0,20):
    batch = get_next_batch(valid_gen)

    choosen_pos = 7

    predictions = ft_model.predict(batch[0])
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(15,5)
    ax1.imshow(np.uint8(batch[0][choosen_pos]))
    ax1.set_title('True label: '+ labels[np.argmax(batch[-1][choosen_pos])])
    ax2.barh(list(labels), predictions[choosen_pos], color=plt.get_cmap('Paired').colors)
    ax2.set_title('Predicted label: '+labels[np.argmax(predictions[choosen_pos])])
    ax2.grid(alpha=.3)
    plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
true_label = batch[1]
pred_label = ft_model.predict(batch[0])
for i in range(0,500):
    batch = get_next_batch(valid_gen)
    true_label = np.concatenate((true_label, batch[1]), axis=0)
    pred_label = np.concatenate((pred_label, ft_model.predict(batch[0])), axis=0)

In [ ]:
#confusion matrix
cm = confusion_matrix(np.argmax(true_label, axis=-1), np.argmax(pred_label, axis=-1))
# Plot the confusion matrix
plt.figure(figsize=(8,8))
sns.heatmap(cm/sum(true_label), xticklabels=labels, yticklabels=labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(tl_history['loss'],label = 'Training Loss', alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Validation Loss', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))

plt.plot(tl_history['accuracy'],label = 'Training Accuracy' , alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Validation accuracy', alpha=.8, color='#4D61E2')
plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()